Allows the notebook to load your username and password from the .env file.

In [1]:
%load_ext dotenv
%dotenv

### Import dependancies and connect to peloton.

You should see a 200 status if it's connected successfully.

In [2]:
import requests
import pandas as pd
import json
import os
import plotly.express as px
from datetime import datetime

s = requests.Session()
payload = {'username_or_email': os.getenv("peloton-username"), 'password': os.getenv("peloton-password")}
s.post('https://api.onepeloton.com/auth/login', json=payload)

<Response [200]>

## Load user data from api

In [3]:
url = 'https://api.onepeloton.com/api/me'
# getting data from source
data = s.get(url).json()

pretty_data = json.dumps(data, indent=4)

print(pretty_data)

{
    "referrals_made": 1,
    "is_strava_authenticated": true,
    "weight": 251.0,
    "first_name": "Angela",
    "block_explicit": false,
    "paired_devices": [
        {
            "name": "RHYTHM+2142737",
            "paired_device_type": "heart_rate_monitor",
            "serial_number": "ED:8A:0D:F0:3F:DF"
        }
    ],
    "estimated_cycling_ftp": 190,
    "customized_heart_rate_zones": [],
    "referral_code": null,
    "last_workout_at": 1603041305,
    "total_non_pedaling_metric_workouts": 85,
    "has_active_device_subscription": true,
    "subscription_credits_used": 0,
    "is_profile_private": false,
    "height": 64.0,
    "last_name": "Wolff",
    "cycling_ftp": 0,
    "is_external_beta_tester": false,
    "instructor_id": null,
    "has_active_digital_subscription": false,
    "phone_number": "7545249439",
    "birthday": 48470400,
    "name": "Angela Wolff",
    "quick_hits": {
        "quick_hits_enabled": true,
        "speed_shortcuts": null,
        "incli

Show your peloton_id.

In [4]:
peloton_id = data['id']
print(peloton_id)

5272efe184544d80999c83364c9554bf


Build the core DataFrame from workoutdata.

In [5]:
df = pd.DataFrame(columns=["percentile", 
                            "workout_date", 
                            "difficulty_rating_avg", 
                            "duration_mins", 
                            "class_length",
                            "total_work"])

races = 50


def get_workout(workout_id: str):
    url = 'https://api.onepeloton.com/api/workout/' + workout_id
    # getting data from source
    data = s.get(url).json()
    
    leaderboard_rank = data['leaderboard_rank']
    total_leaderboard_users= data['total_leaderboard_users']
    avg_pos = (leaderboard_rank / total_leaderboard_users)
    timestamp = datetime.fromtimestamp(data['created_at'])
    difficulty_rating_avg = data['ride']['difficulty_rating_avg']
    pdd = (data['ride']['pedaling_duration'] / 60)
    total_work = data['total_work']/ 1000
    new_row = pd.Series({"percentile": avg_pos, 
                         "workout_date": timestamp, 
                         "difficulty_rating_avg": difficulty_rating_avg, 
                         "duration_mins": pdd, 
                         "class_length": '{0} mins'.format(int(pdd)),
                         "total_work": total_work})
    global df
    df = df.append(new_row, ignore_index=True)


def get_session_data(session_type: str):
    for w in get_data.json()['data']:
        if w['ride']['difficulty_level'] !=  "beginner":
            if w['fitness_discipline'] == session_type:
                get_workout(w['id'])
 

url = 'https://api.onepeloton.com/api/user/{0}/workouts/?joins=ride,ride.instructor&limit={1}&page=0'.format(peloton_id, races)
# getting data from source
get_data = s.get(url)
get_session_data('cycling')


df['workout_date'] = pd.to_datetime(df['workout_date'])

In [6]:
print(df.head())

   percentile        workout_date  difficulty_rating_avg  duration_mins  \
0    0.833205 2020-12-27 11:43:21                 7.6513           30.0   
1    0.840894 2020-12-07 18:21:43                 6.9272           20.0   
2    0.500141 2020-11-24 17:34:35                 5.5136           20.0   
3    0.674268 2020-11-23 22:29:24                 7.4880           20.0   
4    0.700144 2020-11-19 18:09:17                 8.3035           30.0   

  class_length  total_work  
0      30 mins   146.40964  
1      20 mins    84.63181  
2      20 mins   101.63987  
3      20 mins   108.84294  
4      30 mins   151.14779  


## Build a scatter plot

In [8]:
def plot_workout_percentile():
    fig1 = px.scatter(df, x="workout_date", 
                         y="percentile", 
                         title='Workout % position',  
                         trendline="ols", 
                         trendline_color_override="red",
                    )
    fig1.update_yaxes(range=[0, 1])
    fig1.show()

plot_workout_percentile()